In [1]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations, product
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
from sympy import Point, Polygon
from decorators import *
from smartprint import smartprint as sprint
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))
    
SERVER_CACHE_DIR = '/mnt/ci-nas-cache/edulyzeV2/cache_compute_4/fixed_face'
os.makedirs(SERVER_CACHE_DIR,exist_ok=True)

track_analysis_meta_cache = f'{SERVER_CACHE_DIR}/analysis_tracking/meta_info'
base_dir = '/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face/'

track_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_tracking/session_tracking_info'
os.makedirs(track_analysis_session_data,exist_ok=True)

postprocessed_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/processed_id_maps'
os.makedirs(postprocessed_id_map_data_dir, exist_ok=True)

emb_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_emb/session_emb_info_new'
os.makedirs(emb_analysis_session_data,exist_ok=True)

reid_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_emb/session_reid_info'
os.makedirs(reid_analysis_session_data,exist_ok=True)


embmatched_id_raw_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/embmatched_id_raw'
os.makedirs(embmatched_id_raw_data_dir,exist_ok=True)

embmatched_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/embmatched_id_maps_new'
os.makedirs(embmatched_id_map_data_dir, exist_ok=True)

cross_session_input_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/cross_session_input'
os.makedirs(cross_session_input_data_dir, exist_ok=True)

reid_cross_session_input_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/reid_cross_session_input'
os.makedirs(reid_cross_session_input_data_dir, exist_ok=True)

id_viz_cache_root = f'{SERVER_CACHE_DIR}/analysis_emb/session_matching_info'
os.makedirs(id_viz_cache_root, exist_ok=True)

## Get frame file data for all sessions

session_filter_list = [
 'classinsight-cmu_05748A_ghc_4101_201902141630',
 'classinsight-cmu_05748A_ghc_4101_201904021630',
 'classinsight-cmu_05748A_ghc_4101_201902051630',
 'classinsight-cmu_05748A_ghc_4101_201902281630',
 'classinsight-cmu_05748A_ghc_4101_201903071630',
 'classinsight-cmu_17214B_ph_a21_201902271030',
 'classinsight-cmu_17214B_ph_a21_201903061030',
 'classinsight-cmu_17214B_ph_a21_201904031030',
 'classinsight-cmu_17214B_ph_a21_201904101030',
 'classinsight-cmu_17214B_ph_a21_201904241030',
 'classinsight-cmu_17214C_ph_225b_201903201130',
 'classinsight-cmu_17214C_ph_225b_201904101130',
 'classinsight-cmu_17214C_ph_225b_201904171130',
 'classinsight-cmu_17214C_ph_225b_201904241130',
 'classinsight-cmu_17214C_ph_225b_201905011130',
]

In [3]:

frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]        
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
frame_file_data.keys()

dict_keys(['05748A', '05410B', '17214C', '17214B', '05681A', '24352A', '24352C', '86375A'])

In [4]:
frame_file_data['17214C'].keys()

dict_keys(['classinsight-cmu_17214C_ph_225b_201903201130-front', 'classinsight-cmu_17214C_ph_225b_201904031130-front', 'classinsight-cmu_17214C_ph_225b_201904101130-front', 'classinsight-cmu_17214C_ph_225b_201904171130-front', 'classinsight-cmu_17214C_ph_225b_201904241130-front', 'classinsight-cmu_17214C_ph_225b_201905011130-front'])

In [5]:
# get reid for all ids

for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        if session_id.split("-front")[0] not in session_filter_list:
            print(f"Session {session_id} not in session filter list, skipping...")
            continue
        session_reid_cache_file = f"{reid_analysis_session_data}/{session_id}.pb"
        try:
            if not os.path.exists(session_reid_cache_file):
                session_dir = frame_file_data[course][session_id]['dir_location']
                reid_dir = session_dir.replace('fixed_face','reid')
                sprint(reid_dir)
                frame_ids = frame_file_data[course][session_id]['frame_ids']
                session_emb_info = {}
                for frame_id in frame_ids:
                    frame_number, frame_data = pickle.load(open(f'{reid_dir}/{frame_id}.pb','rb'))
                    frame_emb_info = {int(person_info['track_id']):{
                        'bbox': person_info['bbox'] if 'bbox' in person_info else None,
                        'reid_embedding': person_info['reid_features'] if 'reid_features' in person_info else None,
                    } for person_info in frame_data}
                    session_emb_info[frame_id] = frame_emb_info
                pickle.dump(session_emb_info, open(session_reid_cache_file,'wb'))
                print(f"Got emb info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            else:
                ...
                print(f"FILE EXISTS: emb info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
        except:
            print(f"ERROR: Unable to get session emb for: {course_idx}-{course}, session:{session_idx}-{session_id}")
            unfinished_sessions.append((course, session_id))
            print(traceback.format_exc())
    

FILE EXISTS: emb info for session: 0-05748A, session:0-classinsight-cmu_05748A_ghc_4101_201902051630-front
FILE EXISTS: emb info for session: 0-05748A, session:1-classinsight-cmu_05748A_ghc_4101_201902141630-front
Session classinsight-cmu_05748A_ghc_4101_201901291630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201901311630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902071630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902121630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902191630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902211630-front not in session filter list, skipping...
FILE EXISTS: emb info for session: 0-05748A, session:8-classinsight-cmu_05748A_ghc_4101_201902281630-front
Session classinsight-cmu_05748A_ghc_4101_201903051630-front not in session filt

In [7]:
# writing a generic loop to get embedding info from all courses in frame file data
# MAX_GAZE_DEVIATION_DEG = 30
# MIN_EMBEDDING_FRAMES = 100
# CLU_EPS = 0.4
# CLU_MIN_PTS = 100
# np.random.seed(42)

for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        if session_id.split("-front")[0] not in session_filter_list:
            print(f"Session {session_id} not in session filter list, skipping...")
            continue
        cross_session_input_cache_file = f"{reid_cross_session_input_data_dir}/{session_id}.pb"
        try:
            if not os.path.exists(cross_session_input_cache_file):
                printm(f"## Building cross session input info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
                session_emb_info = pickle.load(open(f'{reid_analysis_session_data}/{session_id}.pb','rb'))
                session_id_map = pickle.load(open(f"{postprocessed_id_map_data_dir}/{session_id}.pb","rb"))
                df_session_eligible_pairs = pd.read_csv(f"{embmatched_id_map_data_dir}/{session_id}.csv")
                
                # Correct new ids with dict from eligible pairs
                eligible_id_map_dict = {}
                for id_pair in df_session_eligible_pairs.id_pair.values:
                    (id1, id2) = eval(id_pair)
                    print(id1, id2)
                    if id2 in eligible_id_map_dict:
                        eligible_id_map_dict[id1] = eligible_id_map_dict[id2]
                    else:
                        eligible_id_map_dict[id2] = id1
                
                sprint(eligible_id_map_dict)

                # correct created eligible map for once more
                for key in sorted(list(eligible_id_map_dict.keys())):
                    key_value = eligible_id_map_dict[key]
                    if key_value in eligible_id_map_dict.keys():
                        eligible_id_map_dict[key] = eligible_id_map_dict[key_value]
                sprint("Corrected", eligible_id_map_dict) 

                sprint("replacing ids in old to new id maps")
                for old_id in session_id_map:
                    if session_id_map[old_id] in eligible_id_map_dict:
                        print(f"replacing {old_id}:{session_id_map[old_id]} -->{eligible_id_map_dict[session_id_map[old_id]]}")
                        session_id_map[old_id] = eligible_id_map_dict[session_id_map[old_id]]
                # sprint({kr:old_to_new_id_map[kr] for kr in old_to_new_id_map if (not old_to_new_id_map[kr]==10000)}) 

                # Replace raw ids with mapped ids after postprocessing for both sessions
                session_emb_info = {
                    xr:{
                        session_id_map[yr]:session_emb_info[xr][yr] 
                            for yr in session_emb_info[xr] if not (session_id_map[yr]==10000)} for xr in session_emb_info}

                # arrange info as per tracking id across both sessions
                bbox_info = {}
                emb_info = {}
                for frame_number in session_emb_info:
                    for trackId in session_emb_info[frame_number]:
                        # trackId = trackId_old if (trackId_old not in eligible_id_map_dict.keys()) else eligible_id_map_dict[trackId_old]
                        if trackId not in emb_info:
                            emb_info[trackId]=[]
                            bbox_info[trackId] = []
                        # get  gaze info
                        try:
                            id_bbox = session_emb_info[frame_number][trackId]['bbox']
                            bbox_info[trackId].append([frame_number]+list(id_bbox))
                            body_emb = session_emb_info[frame_number][trackId]['reid_embedding'].tolist()
                            emb_info[trackId].append([frame_number]+body_emb)
                        except:
                            continue
                
                for id in emb_info:
                    emb_info[id] =pd.DataFrame(emb_info[id], columns=['frame']+np.arange(512).tolist()).set_index('frame')
                    bbox_info[id] = pd.DataFrame(bbox_info[id], columns=['frame']+np.arange(5).tolist()).set_index('frame')

                
                sprint({xr:(emb_info[xr].shape[0], bbox_info[xr].shape[0]) for xr in emb_info})

                # get cluster based embeddings
                cluster_based_emb = {}
                for sid in emb_info:
                    cluster_based_emb[sid] = np.median(emb_info[sid],axis=0)
                    sprint(f"Got reid embedding for {sid}")

                # get complete information needed for a given id for cross-session id tracking
                session_id_info = {}
                for sid in sorted(emb_info.keys()):
                    sample_df =deepcopy(bbox_info[sid])
                    
                    sample_df['bbox_width'] = sample_df[2]-sample_df[0]
                    sample_df['bbox_height'] = sample_df[3]-sample_df[1]
                    sample_df['bbox_x'] = (sample_df[2]+sample_df[0]) / 2
                    sample_df['bbox_y'] = (sample_df[3]+sample_df[1]) / 2

                    bbox_width_med, bbox_height_med = sample_df['bbox_width'].median(), sample_df['bbox_height'].median()
                    bbox_width_iqd = sample_df['bbox_width'].quantile(0.75) - sample_df['bbox_width'].quantile(0.25)
                    bbox_height_iqd = sample_df['bbox_height'].quantile(0.75) - sample_df['bbox_height'].quantile(0.25)
                    bbox_x_med, bbox_y_med = sample_df['bbox_x'].median(), sample_df['bbox_x'].median()
                    bbox_x_iqd = sample_df['bbox_x'].quantile(0.75) - sample_df['bbox_x'].quantile(0.25)
                    bbox_y_iqd = sample_df['bbox_y'].quantile(0.75) - sample_df['bbox_y'].quantile(0.25)

                    session_id_info[sid] = dict(bbox_width_med=bbox_width_med, bbox_height_med=bbox_height_med,bbox_width_iqd=bbox_width_iqd, bbox_height_iqd=bbox_height_iqd,
                                         bbox_x_med=bbox_x_med, bbox_y_med=bbox_y_med, bbox_x_iqd=bbox_x_iqd, bbox_y_iqd=bbox_y_iqd, 
                                         cluster_emb = cluster_based_emb.get(sid, None))
                
                # sprint(sid, face_width_med, face_width_iqd, face_height_med, face_height_iqd, face_x_med, face_x_iqd, face_y_med, face_y_iqd)
                pickle.dump(session_id_info, open(cross_session_input_cache_file,'wb'))
                printm(f"### Got cross session input info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            else:
                ...
                printm(f"## FILE EXISTS: cross session input info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
        except:
            print(f"ERROR: Unable to get cross session input info for: {course_idx}-{course}, session:{session_idx}-{session_id}")
            print(traceback.format_exc())
    

## Building cross session input info for session: 0-05748A, session:0-classinsight-cmu_05748A_ghc_4101_201902051630-front

10 32
11 30
24 32
26 32
Dict: eligible_id_map_dict
Key: Value


{24: 10, 26: 10, 30: 11, 32: 10}

"Corrected", eligible_id_map_dict : Corrected {32: 10, 30: 11, 24: 10, 26: 10}
"replacing ids in old to new id maps" : replacing ids in old to new id maps
replacing 519:24 -->10
replacing 649:26 -->10
replacing 697:30 -->11
replacing 1092:32 -->10
replacing 692:26 -->10
replacing 713:26 -->10
replacing 758:26 -->10
replacing 821:26 -->10
replacing 1006:26 -->10
replacing 1162:32 -->10
replacing 1220:32 -->10
replacing 1287:32 -->10
Dict: {xr:(emb_info[xr].shape[0], bbox_info[xr].shape[0]) for xr in emb_info}
Key: Value


{0: (11922, 11922),
 1: (11922, 11922),
 2: (11922, 11922),
 3: (11922, 11922),
 4: (578, 578),
 5: (11922, 11922),
 6: (11922, 11922),
 7: (11922, 11922),
 8: (11920, 11920),
 9: (11922, 11922),
 10: (10782, 10782),
 11: (11875, 11875),
 12: (11773, 11773),
 13: (11603, 11603),
 14: (11323, 11323),
 15: (11343, 11343),
 16: (11173, 11173),
 17: (204, 204),
 18: (165, 165),
 19: (10103, 10103),
 20: (664, 669),
 21: (8383, 8383),
 22: (242, 242),
 23: (570, 570),
 25: (465, 465),
 27: (206, 206),
 28: (168, 168),
 29: (293, 293),
 31: (476, 476),
 33: (241, 241),
 34: (447, 447)}

...

"Corrected", eligible_id_map_dict : Corrected {}
"replacing ids in old to new id maps" : replacing ids in old to new id maps
Dict: {xr:(emb_info[xr].shape[0], bbox_info[xr].shape[0]) for xr in emb_info}
Key: Value


{0: (12144, 12144),
 1: (14984, 14984),
 2: (753, 815),
 3: (3323, 3328),
 4: (14792, 14796),
 5: (14621, 14626),
 6: (14258, 14339),
 7: (13692, 13698),
 8: (13189, 13194),
 9: (12120, 12126),
 10: (1358, 3329),
 11: (11081, 11084),
 12: (10637, 10638),
 13: (16, 920),
 14: (466, 466),
 15: (1726, 1726),
 16: (1938, 1938),
 17: (443, 443),
 18: (430, 430),
 19: (697, 697),
 20: (2735, 2735),
 21: (415, 415)}

f"Got reid embedding for {sid}" : Got reid embedding for 0
f"Got reid embedding for {sid}" : Got reid embedding for 1
f"Got reid embedding for {sid}" : Got reid embedding for 2
f"Got reid embedding for {sid}" : Got reid embedding for 3
f"Got reid embedding for {sid}" : Got reid embedding for 4
f"Got reid embedding for {sid}" : Got reid embedding for 5
f"Got reid embedding for {sid}" : Got reid embedding for 6
f"Got reid embedding for {sid}" : Got reid embedding for 7
f"Got reid embedding for {sid}" : Got reid embedding for 8
f"Got reid embedding for {sid}" : Got reid embedding for 9
f"Got reid embedding for {sid}" : Got reid embedding for 10
f"Got reid embedding for {sid}" : Got reid embedding for 11
f"Got reid embedding for {sid}" : Got reid embedding for 12
f"Got reid embedding for {sid}" : Got reid embedding for 13
f"Got reid embedding for {sid}" : Got reid embedding for 14
f"Got reid embedding for {sid}" : Got reid embedding for 15
f"Got reid embedding for {sid}" : Got reid embeddi

### Got cross session input info for session: 3-17214B, session:9-classinsight-cmu_17214B_ph_a21_201904241030-front

Session classinsight-cmu_17214B_ph_a21_201905011030-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201901301630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902061630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902131633-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902201630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902271630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902271718-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201903131630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201903201630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201903271630-front not in session filter list, skipping...
Se